In [1]:
import findspark
findspark.init("/opt/manual/spark")
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder \
.master("local[2]") \
.appName("Caching and Persistence of Data") \
.getOrCreate()

<h2>Caching and Persistence of Data</h2>

<p>What is the difference between caching and persistence? In Spark <strong>they are synonymous</strong>.</p>
<p>Two API calls, cache() and persist(), offer these capabilities. persistence provides
more control over how and where your data is stored—in memory and on disk,
serialized and unserialized.</p>
<p>Both contribute to better performance for frequently
accessed DataFrames or tables.</p>
With cache(), you use only the default storage level :

    MEMORY_ONLY for RDD
    MEMORY_AND_DISK for Dataset


<img src="../images/dataframe_persist_storage_level.png" />

Source: Learning Spark, O'Reilly, 2020

In [2]:
df = spark.range(1 * 10000000).toDF("id").withColumn("square", F.col("id") * F.col("id"))

In [3]:
df.show(3)

+---+------+
| id|square|
+---+------+
|  0|     0|
|  1|     1|
|  2|     4|
+---+------+
only showing top 3 rows



In [4]:
import datetime as dt

In [5]:
start = dt.datetime.now()
df.cache()
df.count() # Materialize the cache
stop = dt.datetime.now()
print( (stop - start).total_seconds())

11.970594


In [6]:
start = dt.datetime.now()
df.count()
stop = dt.datetime.now()
print( (stop - start).total_seconds())

0.472341


<img src="../images/spark_ui_storage_cached_data.png"  />

## chache() works with the first action.

# 2. persist

In [7]:
df2 = spark.range(1 * 20000000).toDF("id").withColumn("square", F.col("id") * F.col("id"))

In [8]:
from pyspark.storagelevel import StorageLevel

In [9]:
df2.persist(StorageLevel.DISK_ONLY) # Serialize the data and cache it on disk

DataFrame[id: bigint, square: bigint]

In [10]:
start = dt.datetime.now()
df2.count() # materialize
stop = dt.datetime.now()
print( (stop - start).total_seconds())

18.233475


In [11]:
start = dt.datetime.now()
df2.count() # Now get it from the cache
stop = dt.datetime.now()
print( (stop - start).total_seconds())

0.871334


<img src="../images/spark_persist_ui_size_on_disk.png" />

<p>Pay attention that df2 is cached on disk while df is on memory</p>

In [11]:
# To unpersist your cached data

In [12]:
df2.unpersist()

DataFrame[id: bigint, square: bigint]

In [14]:
df.unpersist()

DataFrame[id: bigint, square: bigint]

## DataFrame.persist()

<p>persist(StorageLevel.LEVEL) is nuanced, providing control over how your data is 
cached via StorageLevel</p>

In [13]:
spark.stop()